In [7]:
from pathlib import Path

PROJECT_ROOT = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd().parent
INPUT_FILE = PROJECT_ROOT / "output-data/questions.json"
OUTPUT_FILE = PROJECT_ROOT / "output-data/questions_keyword.json"

print(INPUT_FILE)

/Users/esomin/runtorun/output-data/questions.json


## From string arry input

In [28]:
import re
import json
import unicodedata

# 파일 경로
input_file = "data/questions.json"
output_file = "data/questions_keyword.json"

# 정규식 패턴: "에 대한/에 관한" + 문제 꼬리 어미까지 포함
pattern = re.compile(r"""
    (.+?)                   # 캡처: 핵심 키워드
    (?=                     # Lookahead 시작
        에\s+(대한|관한)       # "에 대한/에 관한"
        | [과와]\s*관련(된|한|하여) # "과/와 관련된/관련한/관련하여"
        | 의\s*내용\s*중        # "의 내용 중"
        | 에\s*해당(하는|하지)   # "에 해당하는/에 해당하지"
        | 로만\s*묶은          # "로만 묶은"
        | 으로                 # "으로"
    )                       # Lookahead 끝
""", re.VERBOSE)

def normalize_text(text: str) -> str:
    """유니코드 정규화"""
    return unicodedata.normalize("NFKC", text).strip()

def clean_prefix(text: str) -> str:
    text = normalize_text(text)
    # 1. [숫자] 문제번호 제거
    text = re.sub(r"^\[\d+\]\s*", "", text)
    # 2. "다음", "다음 중" 제거 (맨 앞 또는 카테고리 뒤)
    text = re.sub(r"(^|\]\s*)다음\s*중\s*", r"\1", text)
    text = re.sub(r"(^|\]\s*)다음\s*", r"\1", text)
    return text.strip()


def extract_subject(stem: str) -> str:
    stem = normalize_text(stem)
    match = pattern.search(stem)
    if match:
        subject = match.group(1).strip()
    else:
        subject = stem
    return clean_prefix(subject)

# 파일 읽기
with open(input_file, "r", encoding="utf-8") as f:
    stems = json.load(f)

# keyword만 리스트로 저장
keywords = [extract_subject(stem) for stem in stems]

# 결과 저장
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(keywords, f, ensure_ascii=False, indent=2)

print(f"✅ 키워드 추출 완료 → {output_file}")


✅ 키워드 추출 완료 → data/questions_keyword.json
